Paramètres à estimer :
1) hyperparamètres : 
    - M nombre de documents
    - K nombre de topics
    - W taille du vocabulaire
    - Psi (décrit en dessous)
    - N[i] nombre de mots par document, i allant de 1 à M (peut suivre une loi de Poisson de paramètre Psi)
    - alpha : vecteur de dimension K correspondant au paramètre de la loi de Dirichlet (cf. exemple des ficelles, où les ficelles correspondent aux documents et les topics à leur découpe. alpha est alors le paramètre qui donne les proportions moyennes de chaque topic pour chaque document. alpha est le prior de Dirichlet placé sur theta
    - beta : prior de Dirichlet placé sur fi
    - fi : matrice de taille K*W, où fi[i,j] désigne la probabilité d'un mot w[j] d'appartenir au topic z[i]
   
2) paramètres latents (qui dépendent des hyperparamètres) : pour chaque document :
    - theta : proportion exacte des topics dans chaque document (de dimension M*K)
    - z : topics associés à chaque mot (de dimension N[i])

Estimation des paramètres : on cherche à estimer z, theta et fi. On suppose que alpha et beta sont connus, ainsi que M, K, W, Psi et les N[i]
    3 méthodes :
        a) variational Bayes (VB)
        b) expectation propagation
        c) collapse Gibbs sampling

In [1]:
# import nltk
# nltk.download()

In [1]:
import os
os.chdir('C:\\Users\\Samir\\Downloads')

In [2]:
import numpy as np

class LDA_Model(object):
    def __init__(self, K, alpha, beta):
                 #, fi, theta, z, clean_corp, unique_corpus, n_jkw):
        """constructor
        self, M, W, Psi, N, alpha, beta : les paramètres fixes de notre modèle"""
        import numpy as np
        import math
        #Paramètres fixes du modèle
        self.M = 0
        self.K = K
        self.W = 0
        #self.Psi = Psi
        #nombre de mots par document
        self.N = np.zeros(self.M)
        #paramètre de Dirichlet sur theta
        #self.alpha = np.zeros(K)
        self.alpha = alpha
        #paramètre de Dirichlet sur fi
        self.beta = beta
        #theta
        self.theta = np.zeros((self.M, K))
        #fi
        self.fi = np.zeros((K,self.W))
        #z
        self.z = [[] for _ in range(self.M)]
        #on définit aussi un corpus nettoyé vide
        self.clean_corp = []
        #on définit les uniques mots du corpus pour chaque document
        self.unique_corpus = [[] for _ in range(self.M)]
        #le corpus entier en 1 liste
        self.unique_flattened_corpus = []
        #self.n_jkw = [[[] for k in range(self.K)] for j in range(self.M)]
        self.n_jkw = [[{} for k in range(self.K)] for j in range(self.M)]

In [3]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string
from nltk.stem.porter import PorterStemmer
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
import glob


#corpus représente le corpus de documents dont on cherche à appliquer la LDA
#on va d'abord nettoyer le corpus
def clean_corpus(self, corpus):
    clean_corp = self.clean_corp
    files = glob.glob(corpus)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    porter = PorterStemmer()
#     snowball = SnowballStemmer('english')
#     wordnet = WordNetLemmatizer()
#on itère sur le corpus pour récupérer tous les fichiers
    rootdir = 'BBC'
    for subdir, dirs, files in os.walk(rootdir):
        for fle in files:
       #on lit tous les fichiers
            with open(subdir+"\\"+fle) as f:
                text = f.read()
                #pour chaque document du corpus, on effectue une tokenisation
                tokenized_text = word_tokenize(text)
                #on supprime la ponctuation
                tokenized_docs_no_punctuation = []
                for token in tokenized_text: 
                    new_token = regex.sub(u'', token)
                    if not new_token == u'':
                        tokenized_docs_no_punctuation.append(new_token)
                #on supprime les "stop words"
                tokenized_docs_no_stopwords = []
                for word in tokenized_docs_no_punctuation:
                    if not word in stopwords.words('english'):
                        tokenized_docs_no_stopwords.append(word)
                #enfin, on concatène les mots ayant des similarités de sens
                preprocessed_docs = []
                for word in tokenized_docs_no_stopwords:
                    preprocessed_docs.append(porter.stem(word))
                #on supprime les mots de moins de 3 caractères
                out_length_words = []
                for word in preprocessed_docs:
                    if(len(word)>3):
                        out_length_words.append(word)
                clean_corp.append(out_length_words)    
        #on retourne le corpus nettoyé, qui est une liste de chaque document du corpus
        #return clean_corp

In [4]:
#ici, on va définir le nombre de documents M, le nombre de mots par documents N[i], ainsi que
#la taille du vocabulaire W
def define_hyperparams_M_N_W(self):
    #on définit le nombre de documents M
    self.M = len(self.clean_corp)
    #vecteur représentant le nombre de mots par document
    #self.N = N
    clean_corp = self.clean_corp
    #on définit les uniques mots du corpus par document
    unique_corpus = [[] for _ in range(self.M)]
    #maintenant on va définir la taille du vocabulaire
    #on transforme le corpus en une liste
    flattened_corpus = [y for x in self.clean_corp for y in x]
    unique_flattened_corpus = self.unique_flattened_corpus
    for i in flattened_corpus:
        if i not in unique_flattened_corpus:
            unique_flattened_corpus.append(i)
    W = len(unique_flattened_corpus)
    self.W = W
    self.unique_flattened_corpus = unique_flattened_corpus
    #on définit aussi les mots uniques par document
    i1 = 0
    for i in clean_corp:
        for j in i:
            if j not in unique_corpus[i1]:
                unique_corpus[i1].append(j)
        i1 = i1 + 1
    self.unique_corpus = unique_corpus
    self.W = W

In [5]:
#on définit maintenant alpha, le paramètre de la loi de Dirichlet sur theta
#on suppose K, le nombre de topics, fixé
def define_hyperparams_alpha(self):
    K = self.K
    alpha = np.random.dirichlet(np.ones(K),size=1)[0]
    self.alpha = alpha

In [6]:
#on définit aussi beta, le paramètre de la loi de Dirichlet sur fi
def define_hyperparams_beta(self):
    W = self.W
    beta = np.random.dirichlet(np.ones(W),size=1)[0]
    self.beta = beta

In [7]:
#maintenant, on doit assigner aléatoirement un topic à chaque mot de chaque document
def define_z_init(self):
    K = self.K
    clean_corp = self.clean_corp
    alpha = self.alpha
    z = [[] for _ in range(self.M)]
    for i in range(len(clean_corp)):
        for j in range(len(clean_corp[i])):
            #on génère une loi multinomiale selon les probabilités de Dirichlet de alpha
            #on tire un élément et on associe le topic au mot
            p = np.random.multinomial(1, alpha, size=1)
            z[i].append(np.argmax(p))
    self.z = z

In [8]:
#on compte par mot, le nombre de mots pour chaque topic, pour chaque document
#on obtient une matrice à 3 indices
from collections import Counter
from itertools import compress
def compute_n_jkw(self):
    #unique_corpus = self.unique_corpus
    clean_corp = self.clean_corp
    z = self.z
    M = self.M
    K = self.K
    n_jkw = [[{} for k in range(self.K)] for j in range(self.M)]
    #j document
    for j in range(M):
        #k topic
        for k in range(K):
            #on récupère d'abord les mots pour un document, pour un topic donnée
            fil = [x in [k] for x in test.z[j]]
            words_for_j_and_k = list(compress(clean_corp[j], fil))
            #create a counter and then a dictionary of words in the document for a topic
            dictionary = Counter(words_for_j_and_k)
            n_jkw[j][k].update(dictionary)
    self.n_jkw = n_jkw

In [9]:
#on compute n_jk.-ij
def compute_n_jk_without_ij(self, j, k, w):
    n_jkw = self.n_jkw
    if w in self.n_jkw[j][k]:
        dict_n_jkw = n_jkw[j][k]
        #global dict_without_ij
        dict_without_ij = {key : dict_n_jkw[key] for key in dict_n_jkw if key != w}
        return (sum(dict_without_ij.values()), len(dict_without_ij)+1)
    else:
        return 0

In [10]:
#on compute n_jk.-ij^2 pour avoir l'espérance de la variable au carré
def compute_n_jk_without_ij2(self, j, k, w):
    n_jkw = self.n_jkw
    if w in self.n_jkw[j][k]:
        dict_n_jkw = n_jkw[j][k]
        #global dict_without_ij
        dict_without_ij = {key : dict_n_jkw[key] for key in dict_n_jkw if key != w}
        return (sum([i ** 2 for i in dict_without_ij.values()]), len(dict_without_ij)+1)
    else:
        return 0

In [11]:
#on compute l'espérance de n_jk.-ij par rapport à w
def esperance_n_jk_without_ij(self, j, k, w):
    if w in self.n_jkw[j][k]:
        return compute_n_jk_without_ij(self, j, k, w)[0] / compute_n_jk_without_ij(self, j, k, w)[1]
    else:
        return 0

In [12]:
#on compute l'espérance de n_jk.-ij^2
def esperance_n_jk_without_ij2(self, j, k, w):
    if w in self.n_jkw[j][k]:
        return compute_n_jk_without_ij2(self, j, k, w)[0] / compute_n_jk_without_ij2(self, j, k, w)[1]
    else:
        return 0

In [13]:
#maintenant on compute la variance de n_jk.-ij
def variance_n_jk_without_ij(self, j, k, w):
    if w in self.n_jkw[j][k]:
        return esperance_n_jk_without_ij2(self, j, k, w) - (esperance_n_jk_without_ij(self, j, k, w)*esperance_n_jk_without_ij(self, j, k, w))
    else:
        return 0

In [14]:
#on compute l'espérance de n_.k.-ij par rapport à w et par rapport à j
def esperance_n_k_without_ij(self, j, k, w):
    M = self.M
    s = 0
    for j1 in range(M):
        if w in self.n_jkw[j1][k]:
            s = s + esperance_n_jk_without_ij(self, j1, k, w)
    s = s - esperance_n_jk_without_ij(self, j, k, w)
    s = s / (M - 1)
    return s

In [15]:
#on compute l'espérance de n_.k.-ij^2
def esperance_n_k_without_ij2(self, j, k, w):
    M = self.M
    s = 0
    for j1 in range(M):
        if w in self.n_jkw[j1][k]:
            s = s + esperance_n_jk_without_ij2(self, j1, k, w)
    s = s - esperance_n_jk_without_ij2(self, j, k, w)
    s = s / (M - 1)
    return s

In [16]:
#on compute maintenant la variance de n_.k.-ij^2
def variance_n_k_without_ij(self, j, k, w):
    return esperance_n_k_without_ij2(self, j, k, w) - (esperance_n_k_without_ij(self, j, k, w)**2)

In [17]:
#on compute n_.kx[ij]-ij
def compute_n_kxij_without_ij(self, j, k, w):
    s = 0
    M = self.M
    n_jkw = self.n_jkw
    #on itère selon tous les documents, sauf le j
    for j1 in [x for x in range(M) if x != j]:
        if w in self.n_jkw[j1][k]:
        #on fait la somme de tous les valeurs ayant w comme clé dans le dictionnaire n_jkw[j1][k]
            s = s + n_jkw[j1][k][w]
        else:
            s = 0
    return s

In [18]:
#on compute n_.kx[ij]-ij pour avoir l'espérance de la variable au carré
def compute_n_kxij_without_ij2(self, j, k, w):
    s = 0
    M = self.M
    n_jkw = self.n_jkw
    #on itère selon tous les documents, sauf le j
    for j1 in [x for x in range(self.M) if x != j]:
        if w in self.n_jkw[j1][k]:
        #on fait la somme de tous les valeurs ayant w comme clé dans le dictionnaire n_jkw[j1][k]
            s = s + n_jkw[j1][k][w]
    return s

In [19]:
#on compute l'espérance de n_.kx[ij]-ij
def esperance_n_kxij_without_ij(self, j, k, w):
    M = self.M
    return compute_n_kxij_without_ij(self, j, k, w) / (M-1)

In [20]:
#on compute l'espérance de n_.kx[ij]-ij^2
def esperance_n_kxij_without_ij2(self, j, k, w):
    M = self.M
    return compute_n_kxij_without_ij2(self, j, k, w) / (M-1)

In [21]:
#on compute maintenant la variance de n_.kx[ij]-ij^2
def variance_n_kxij_without_ij(self, j, k, w):
    return esperance_n_kxij_without_ij2(self, j, k, w) - (esperance_n_kxij_without_ij(self, j, k, w)**2)

In [22]:
# test.n_jkw[0][0]

In [23]:
# loc_esp__n_jk_without_ij = [[[]*len(test.n_jkw[0][0]) for w in range(len(test.unique_corpus[j]))] for j in range(test.M)]

In [27]:
# if 'busi' in test.n_jkw[0][0]:
#                      loc_esp__n_jk_without_ij[0][0].append(esperance_n_jk_without_ij(test, 0, 0, 'boost'))

In [28]:
test.n_jkw[0][0]

NameError: name 'test' is not defined

In [35]:
#initialisation des paramètres en utilisant le Collapse Gibbs Sampling
def collapse_Gibbs_sampling(self, nbiter, alpha, beta, K):
    alpha = self.alpha
    beta = self.beta
    K = self.K
    M = self.M
    W = self.W
    n_jkw = self.n_jkw
    unique_corpus = self.unique_corpus
    z = self.z
    #on compute une première fois toutes les espérances requises
    loc_esp__n_jk_without_ij = [[[] for w in range(len(unique_corpus[j]))] for j in range(M)]
    loc_var__n_jk_without_ij = [[[] for w in range(len(unique_corpus[j]))] for j in range(M)]
    loc_esp__n_kxij_without_ij = [[[] for w in range(len(unique_corpus[j]))] for j in range(M)]
    loc_var__n_kxij_without_ij = [[[] for w in range(len(unique_corpus[j]))] for j in range(M)]
    loc_esp__n_k_without_ij = [[[] for w in range(len(unique_corpus[j]))] for j in range(M)]
    loc_var__n_k_without_ij = [[[] for w in range(len(unique_corpus[j]))] for j in range(M)]
    
    print("début de computation des espérances et variances")
    for j in range(M):
        for w in unique_corpus[j]:
            for k in range(K):
                index_w = unique_corpus[j].index(w)
                #on définit des variables locales
                #if w in self.n_jkw[j][k]:
                loc_esp__n_jk_without_ij[j][index_w].append(esperance_n_jk_without_ij(self, j, k, w))
                loc_var__n_jk_without_ij[j][index_w].append(variance_n_jk_without_ij(self, j, k, w))
                loc_esp__n_kxij_without_ij[j][index_w].append(esperance_n_kxij_without_ij(self, j, k, w))
                loc_var__n_kxij_without_ij[j][index_w].append(variance_n_kxij_without_ij(self, j, k, w))
                loc_esp__n_k_without_ij[j][index_w].append(esperance_n_k_without_ij(self, j, k, w))
                loc_var__n_k_without_ij[j][index_w].append(variance_n_k_without_ij(self, j, k, w))
#                 loc_esp__n_jk_without_ij[j][index_w].append(1)
#                 loc_var__n_jk_without_ij[j][index_w].append(1)
#                 loc_esp__n_kxij_without_ij[j][index_w].append(1)
#                 loc_var__n_kxij_without_ij[j][index_w].append(1)
#                 loc_esp__n_k_without_ij[j][index_w].append(1)
#                 loc_var__n_k_without_ij[j][index_w].append(1)
                
    print("fin de computation des espérances et variances")          
    for i in range(nbiter):
        print(i)
        for j in range(M):
            for w in unique_corpus[j]:
            #for k in range(K):
                p = np.zeros(K)
                index_w = unique_corpus[j].index(w)
                k_topic = z[j][index_w]
                loc_esp__n_jk_without_ij[j][index_w][k_topic] -= 1/(len(n_jkw[j][k_topic]))
                loc_var__n_jk_without_ij[j][index_w][k_topic] -= 1/(len(n_jkw[j][k_topic]))
                loc_esp__n_kxij_without_ij[j][index_w][k_topic] -= 1/(M-1)
                loc_var__n_kxij_without_ij[j][index_w][k_topic] -= 1/(M-1)
                loc_esp__n_k_without_ij[j][index_w][k_topic] -= 1/((M-1)*(len(n_jkw[j][k_topic])))
                loc_var__n_k_without_ij[j][index_w][k_topic] -= 1/((M-1)*(len(n_jkw[j][k_topic])))
                        
                a1 = (loc_var__n_jk_without_ij[j][index_w][k_topic])/(2*(0.1 + loc_esp__n_jk_without_ij[j][index_w][k_topic])**2)
                a2 = (loc_var__n_kxij_without_ij[j][index_w][k_topic])/(2*(0.1 + loc_esp__n_kxij_without_ij[j][index_w][k_topic])**2)
                a3 = (loc_var__n_k_without_ij[j][index_w][k_topic])/(2*(W*0.1 + loc_esp__n_k_without_ij[j][index_w][k_topic]))
                exp = np.exp(-a1 -a2 +a3)
                for k in range(K):
                    p[k] = ((0.1 + loc_esp__n_jk_without_ij[j][index_w][k])*(0.1 + loc_esp__n_kxij_without_ij[j][index_w][k])/(0.1*W + loc_esp__n_k_without_ij[j][index_w][k]))*exp
                #on normalise les p[k] calculés ci-dessus
                p_normalized = p/sum(p)
                #on définit alors une nouvelle multinomiale pour le mot w, de longueur K, pour choisir le nouveau topic 
                proba = np.random.multinomial(1, p_normalized, size=1)
                #on détermine le nouveau topic qui a la plus forte probabilité pour le mot w
                new_topic = np.argmax(proba)
                #on remplace l'élément de z et on réactualise espérances et variances
                z[j][index_w] = new_topic
                loc_esp__n_jk_without_ij[j][index_w][new_topic] += 1/(len(n_jkw[j][new_topic]))
                loc_var__n_jk_without_ij[j][index_w][new_topic] += 1/(len(n_jkw[j][new_topic]))
                loc_esp__n_kxij_without_ij[j][index_w][new_topic] += 1/(M-1)
                loc_var__n_kxij_without_ij[j][index_w][new_topic] += 1/(M-1)
                loc_esp__n_k_without_ij[j][index_w][new_topic] += 1/((M-1)*(len(n_jkw[j][new_topic])))
                loc_var__n_k_without_ij[j][index_w][new_topic] += 1/((M-1)*(len(n_jkw[j][new_topic])))

In [25]:
#on implémente alors les paramètres fi et theta
#on compute d'abord toutes les espérances nécessaires
#espérance de n_jk.
def compute_n_jk(self, j, k):
    n_jkw = self.n_jkw
    dict_n_jkw = n_jkw[j][k]
    return (sum(dict_n_jkw.values()), len(dict_n_jkw))

def esperance_n_jk(self, j, k):
    return compute_n_jk(self, j, k)[0] / compute_n_jk(self, j, k)[1]

#espérance de n_.kw
def compute_n_kxij(self, k, w):
    s = 0
    n_jkw = self.n_jkw
    M = self.M
    #for j1 in [x for x in range(len(n_jkw))]:
    for j1 in range(M):
        if w in self.n_jkw[j1][k]:
            s = s + n_jkw[j1][k][w]
    return s

def esperance_n_kxij(self, k, w):
    M = self.M
    return compute_n_kxij(self, k, w) / M

#espérance de n_.k.
def esperance_n_k(self, k):
    M = self.M
    s = 0
    for j1 in range(M):
        s = s + esperance_n_jk(self, j1, k)
    s = s / M
    return s
    
#espérance de n_j..
def esperance_n_j(self, j):
    K = self.K
    s = 0
    for k1 in range(K):
        s = s + esperance_n_jk(self, j, k1)
    s = s / K
    return s

In [26]:
#maintenant on compute theta et fi
def compute_theta(self):
    alpha = self.alpha
    K = self.K
    M = self.M
    theta = np.zeros((self.M, K))
    for j in range(M):
        for k in range(K):
            #theta[j,k] = (alpha[k] + esperance_n_jk(self, j, k)) / (K*alpha[k] + esperance_n_j(self, j))
            theta[j,k] = (0.1 + esperance_n_jk(self, j, k)) / (K*0.1 + esperance_n_j(self, j))
    self.theta = theta
            
def compute_fi(self):
    beta = self.beta
    W = self.W
    K = self.K
    unique_corpus = self.unique_corpus
    unique_flattened_corpus = self.unique_flattened_corpus
    fi = np.zeros((K,self.W))
    for k in range(K):
        for w1 in range(len(unique_flattened_corpus)):
            w = unique_flattened_corpus[w1]
            fi[k,w1] = (0.1 + esperance_n_kxij(self, k, w)) / (W*0.1 + esperance_n_k(self, k))
    self.fi = fi

In [27]:
#on fait le test de l'algorithme
test = LDA_Model(K = 5, alpha =0.1, beta = 0.1)

In [28]:
clean_corpus(test, "corpus_test/*.txt")

In [29]:
define_hyperparams_M_N_W(test)

In [30]:
define_hyperparams_alpha(test)

In [31]:
define_hyperparams_beta(test)

In [32]:
define_z_init(test)

In [33]:
compute_n_jkw(test)

In [ ]:
collapse_Gibbs_sampling(test, 100, test.alpha, test.beta, test.K)

début de computation des espérances et variances


In [32]:
compute_theta(test)

In [33]:
compute_fi(test)

In [34]:
testo = np.transpose(test.fi)

In [35]:
tall = np.zeros(len(testo))
for i in range(len(testo)):
    tall[i] = np.argmax(testo[i])

In [36]:
fil1 = [x in [0] for x in tall]
topic1 = list(compress(test.unique_flattened_corpus, fil1))

fil2 = [x in [1] for x in tall]
topic2 = list(compress(test.unique_flattened_corpus, fil2))

In [37]:
topic1

['beavertail',
 'sever',
 'confect',
 'whip',
 'cream',
 'sugar',
 'hazelnut',
 'oper',
 'castor',
 'stand',
 'trademark',
 'compani',
 'bell',
 'tree',
 'metr',
 'member',
 'nativ',
 'mexico',
 'central',
 'plant',
 'mountain',
 '4900–5600',
 'leav',
 'dietari',
 'peopl',
 'pedro',
 'tuber',
 'herbac',
 'grow',
 'winter',
 'develop',
 'canelik',
 '4angl',
 'stem',
 'node',
 'larg',
 'pendant',
 '75150mm',
 'floret',
 'colour',
 'speci',
 'growth',
 'link',
 'hour',
 'usual',
 'first',
 'least',
 'horizont',
 'brought',
 'europ',
 '1823–1885',
 'travel',
 'later',
 '1872–73',
 'rhizom',
 'reach',
 'rare',
 'centimet',
 'leaflet',
 'ovat',
 '5–10',
 'head',
 'pink',
 'purpl',
 'fish',
 'brewi',
 'pronounc',
 'brew',
 'newfoundland',
 'hard',
 'with',
 'abund',
 'around',
 'synonym',
 'mani',
 'delicaci',
 'serv',
 'vari',
 'commun',
 'ingredi',
 'alway',
 'typic',
 'salt',
 'soak',
 'water',
 'overnight',
 'reduc',
 'also',
 'next',
 'separ',
 'tender',
 'scrunchion',
 'pork',
 'piec',


In [38]:
topic2

['pastri',
 'similar',
 'dough',
 'choic',
 'sweet',
 'condiment',
 'banana',
 'slice',
 'crumbl',
 'oreo',
 'cinnamon',
 'chocol',
 'canada',
 'franchis',
 'current',
 'store',
 'queue',
 'worldwid',
 'regist',
 'sinc',
 '1988',
 'affili',
 'dahlia',
 'imperiali',
 'tall',
 'genu',
 'america',
 'colombia',
 'upland',
 'occur',
 'elev',
 '1500–1700',
 'supplement',
 'qeqchi',
 'carchá',
 'alta',
 'verapaz',
 'guatemala',
 'perenni',
 'rapidli',
 'base',
 'dormant',
 'period',
 'brittl',
 'swollen',
 'tripinn',
 'near',
 'ground',
 'soon',
 'shed',
 'flowerhead',
 'across',
 'lavend',
 'mauvishpink',
 'fastgrow',
 'spurt',
 'shorter',
 'daylight',
 'come',
 'flower',
 'autumn',
 'frost',
 'propag',
 'seed',
 'long',
 'laid',
 'soil',
 'some',
 '16th',
 'centuri',
 'describ',
 '1863',
 'benedikt',
 'roezl',
 'great',
 'czech',
 'orchid',
 'collector',
 'year',
 'went',
 'odyssey',
 'pinnata',
 'root',
 'height',
 'erect',
 'branch',
 'infloresc',
 'simpl',
 'eight',
 'diamet',
 'length',